# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [21]:
# Imports here
import torch
import numpy as np
import pandas as pd
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import matplotlib.pyplot as plt  
from glob import glob
from tqdm import tqdm
from PIL import Image
import os
import time
import copy

In [22]:
# check if using GPU
gpu_mode = torch.cuda.is_available()
print(gpu_mode)

True


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [23]:
data_dir = 'flower_data'
train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')

In [24]:
# Define batch size
batch_size = 32
# Define transforms for the training and validation sets
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
train_data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        normalize,
    ])

validate_data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])


# Load the datasets with ImageFolder
train_dataset = datasets.ImageFolder(
    train_dir,
    train_data_transforms)

validate_dataset = datasets.ImageFolder(
    valid_dir,
    validate_data_transforms)



# Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=4)

validate_loader = torch.utils.data.DataLoader(
    validate_dataset, batch_size=batch_size, shuffle=True,
    num_workers=4)

data_loader = {}
data_loader['train'] = train_loader
data_loader['valid'] = validate_loader
# print(data_loader)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [25]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [26]:
model_input = 'resnet152'
# Build and train network
if model_input == 'vgg16':
    # Build and train network
    model = models.vgg16(pretrained=True)
elif model_input == 'vgg19':
    # Build and train network
    model = models.vgg19(pretrained=True)
elif model_input == 'resnet152':
    model = models.resnet152(pretrained=True)

# Freeze training for all layers
for param in model.parameters():
    param.requires_grad = False


# # Newly created modules have require_grad=True by default
if 'vgg' in model_input:
    num_features = model.classifier[-1].in_features
    model.classifier[6] = nn.Sequential(
                          nn.Linear(num_features, 512), 
                          nn.ReLU(), 
                          nn.Linear(512, len(cat_to_name)),
                          nn.LogSoftmax(dim=1))
elif 'resnet' in model_input:
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
                            nn.Linear(num_features, 512), 
                              nn.ReLU(), 
                              nn.Dropout(0.4),
                              nn.Linear(512, len(cat_to_name)),
                              nn.LogSoftmax(dim=1))

print(model.__class__.__name__)
# check to see that your last layer produces the expected number of outputs
# print(model.classifier[-1].out_features)

ResNet


In [27]:
# Find total parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

59,245,222 total parameters.
1,101,414 training parameters.


In [28]:
# define the loss function and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
m_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=3,verbose=True)

In [9]:
# function that evaluates model
def evaluate_model(model, criterion, dataset_loader, gpu_mode):
    since = time.time()
    # track test loss
    test_loss = 0.0
    class_correct = list(0. for i in range(len(cat_to_name)))
    class_total = list(0. for i in range(len(cat_to_name)))

    print("Evaluating model")
    print('-' * 10)
    
    model.eval()
    # iterate over test data
    for data, target in dataset_loader:
        # move tensors to GPU if CUDA is available
        if gpu_mode:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update test loss 
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)    
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not gpu_mode else np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(batch_size):
            try:
                label = target.data[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1
            except: pass

    # average test loss
    test_loss = test_loss/len(dataset_loader.dataset)
    print('Test Loss: {:.6f}\n'.format(test_loss))


    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))

In [10]:
# test before training
if gpu_mode:
    model.cuda()
else:
    model.cpu()  

evaluate_model(model, criterion, validate_loader, gpu_mode)

Evaluating model
----------
Test Loss: 4.630502


Test Accuracy (Overall):  1% (16/818)

Evaluation completed in 0m 16s


In [29]:
def train_model(model, criterion, optimizer, scheduler, gpu_mode, data_loader, n_epochs=10):
    # keeping track of best weights
    best_model_wts = copy.deepcopy(model.state_dict())
    valid_loss_min = np.Inf # track change in validation loss

    for epoch in range(1, n_epochs+1):
        # keep track of duration of each epoch
        since = time.time()
        running_losses  = {}
        for phase in ['train', 'valid']:
            # keep track of training and validation loss
            running_loss = 0.0
            valid_acc = 0.0
            
            if phase == 'train':
                # scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            for data, target in data_loader[phase]:
                # move tensors to GPU if CUDA is available
                if gpu_mode:
                    data, target = data.cuda(), target.cuda()
                else:
                    data, target = data, target
                # clear the gradients of all optimized variables
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # forward pass: compute predicted outputs by passing inputs to the model
                    output = model(data)
                    # calculate the batch loss
                    loss = criterion(output, target)
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        # backward pass: compute gradient of the loss with respect to model parameters
                        loss.backward()
                        # perform a single optimization step (parameter update)
                        optimizer.step()
                        
                    # get prediction and accuracy in validate phase
                    else:
                        # convert output probabilities to predicted class
                        _, preds = torch.max(output, 1)    
                        valid_acc += torch.sum(preds == target.data)
                    
                    # update average running loss 
                    running_loss += loss.item()*data.size(0)

            # calculate average losses
            running_losses[phase] = running_loss/len(data_loader[phase].dataset)
            if phase == 'valid':
                # reduce learning rate only when the validation loss doesnt decrease for a bit.
                scheduler.step(running_losses[phase])
        valid_accuracy = float(valid_acc)/float(len(data_loader['valid'].dataset))

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Accuracy: {:.6f} ({}/{})'.format(
            epoch, running_losses['train'], running_losses['valid'], valid_accuracy, valid_acc,len(data_loader['valid'].dataset)))

        # save model if validation loss has decreased
        if running_losses['valid'] <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min, running_losses['valid']))
            best_model_wts = copy.deepcopy(model.state_dict())
            valid_loss_min = running_losses['valid']
        
        elapsed_time = time.time() - since
        print()
        print("Epoch completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
        torch.cuda.empty_cache()
        
    model.load_state_dict(best_model_wts)
    return model

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [32]:
# Save the checkpoint, for VGG16
def save_checkpoint(model, optimizer, scheduler, model_file, class_to_idx):
    model.class_to_idx = class_to_idx
    if 'VGG' in model.__class__.__name__:
        parameters = {
            'class_to_idx': model.class_to_idx,
            'classifier': model.classifier,
            'state_dict': model.state_dict(),
            'optimizer': optimizer,
            'optimizer_state_dict': optimizer.state_dict()
        }
    elif 'ResNet' in model.__class__.__name__:
        parameters = {
            'class_to_idx': model.class_to_idx,
            'classifier': model.fc,
            'state_dict': model.state_dict(),
            'optimizer': optimizer,
            'optimizer_state_dict': optimizer.state_dict()
        }
    
    torch.save(parameters, model_file)

In [33]:
if gpu_mode:
    model.cuda()
else:
    model.cpu() 
model = train_model(model, criterion, optimizer, m_lr_scheduler, gpu_mode, data_loader, n_epochs=1)
save_checkpoint(model, optimizer, lr_scheduler, 'model_flower_classifier_resnet152.pt',train_dataset.class_to_idx)

Epoch: 1 	Training Loss: 4.639208 	Validation Loss: 4.627061 	Validation Accuracy: 0.004890 (4/818)
Validation loss decreased (inf --> 4.627061).  Saving model ...

Epoch completed in 4m 39s


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [40]:
# Function that loads a checkpoint and rebuilds the model (VGG16)
def load_checkpoint(model_file):
    checkpoint = torch.load(model_file)
    if 'vgg19' in model_file:
        model = models.vgg19(pretrained=True)
    elif 'vgg16' in model_file:
        model = models.vgg16(pretrained=True)
    elif 'resnet152' in model_file:
        model = models.resnet152(pretrained=True)
    # freeze parameters
    for param in model.parameters():
        param.requires_grad = False
    
    if 'vgg' in model_file:
        model.classifier = checkpoint['classifier']
    elif 'resnet' in model_file:
        model.fc = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])
    
    # model specifics
    model.class_to_idx = checkpoint['class_to_idx']
    
    # optimizer
    optimizer = checkpoint['optimizer']
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    
    return model, optimizer

In [41]:
model, optimizer = load_checkpoint('model_flower_classifier_resnet152.pt')

In [38]:
if gpu_mode:
    model.cuda()
else:
    model.cpu() 
evaluate_model(model, criterion, validate_loader, gpu_mode)

NameError: name 'evaluate_model' is not defined

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # images have to be normalized
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    # preprocess step
    preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize
        ])
    loaded_image = Image.open(image)
    img_tensor = preprocess(loaded_image).float()
    img_tensor = img_tensor.unsqueeze(0)
    np_image = np.array(img_tensor)
    return np_image


To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model.eval()
    
    # preprocess image
    img_tensor = process_image(image_path)
    img_tensor = torch.from_numpy(img_tensor)
    if gpu_mode:
        img_tensor = img_tensor.cuda()
        model = model.cuda()

    output = model(img_tensor)
    # softmax layer to convert the output to probabilities
    m = nn.Softmax()
    output = m(output)

    probabilities = torch.topk(output,5)[0]
    labels = torch.topk(output,5)[1]
    
    if gpu_mode:
        probabilities = probabilities.cpu()
        labels = labels.cpu()
    
    return img_tensor, probabilities.detach().numpy(), labels.detach().numpy()

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
def display_predict(image_path, model, topk):
    """Display image and preditions from model"""

    # Get predictions
    img, probs, preds = predict(image_path, model, topk)
    
    # get appropriate labels
    idx_to_name = {category: cat_to_name[str(category)] for category in preds[0]}
    classes = list(idx_to_name.values())
    
    if gpu_mode:
        img = img.cpu().squeeze()
        
    # Convert results to dataframe for plotting
    result = pd.DataFrame({'p': probs[0]}, index=classes)

    # Show the image
    plt.figure(figsize=(16, 5))
    ax = plt.subplot(1, 2, 1)
    ax = imshow(img, ax=ax)

    # Set title to be the actual class
    ax.set_title(classes[0], size=20)

    ax = plt.subplot(1, 2, 2)
    # Plot a bar plot of predictions
    result.sort_values('p')['p'].plot.barh(color='blue', edgecolor='k', ax=ax)
    plt.xlabel('Predicted Probability')
    plt.tight_layout()

In [ ]:
# TODO: Display an image along with the top 5 classes
test_img_path = '/home/ec2-user/test_images/image_05188.jpg'
display_predict(test_img_path, model, 5)